In [26]:
import spacy
import pandas as pd
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import unicodedata

nlp = spacy.load("es_core_news_md")

In [27]:
data = pd.read_csv("fake_news.csv")
data

,class,Text
0,True,Algunas de las voces extremistas más conocida...
1,True,Después de casi dos años y medio de luchas po...
2,True,Dos periodistas birmanos de la agencia Reuter...
3,True,El Cuerpo Nacional de Policía ha detenido a c...
4,True,El desfile de la firma en Roma se convierte e...
...,...,...
1995,True,El Consejo de Gobierno ha dado su visto bueno...
1996,True,Investigadores valencianos han desarrollado u...
1997,True,Los arrestados actuaban en coches y en establ...
1998,True,El Rey ha encargado este miércoles a Pedro Sá...


In [37]:
classes = data['class']
y = []
for i in classes:
    if i == True:
        y.append(1)
    else:
        y.append(0)

2000

In [29]:
texts = data['Text'].values.tolist()

In [30]:
def normalize(text):
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    doc = nlp(text)
    tokens = [t for t in doc if t.text.isalpha() and t not in STOP_WORDS and len(t.text) > 2]
    words = []
    for t in tokens:
        words.append(t.text.lower())
    cleaned_text = " ".join(words)
    return cleaned_text


In [35]:
X = []
for i in texts:
    X.append(normalize(i))

2000

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.3,
                                                    random_state=0)

In [45]:
vect = CountVectorizer()
X_train_vectorized = vect.fit_transform(X_train)
X_test_vectorized = vect.transform(X_test)

In [46]:
modelLR = LogisticRegression(solver='liblinear')
modelLR.fit(X_train_vectorized, y_train)

LogisticRegression(solver='liblinear')

In [50]:
y_pred = modelLR.predict(X_test_vectorized)

In [51]:
pd.DataFrame({
    'text':X_test,
    'label':y_test,
    'prediction': pred_y
}).sample(10)

,text,label,prediction
357,six group ofrece euros por accion bme una prim...,1,1
142,para dar voz las personas menos visibles dentr...,1,1
375,segun informacion publicada por diario digital...,0,1
181,protestas los viticultores vilafranca del pene...,1,1
540,adidas regala pares zapatos gratis para celebr...,0,0
72,son labradores los regalan estais interesadas ...,0,0
7,llevar los ninos iglesia vulnera sus derechos ...,0,1
301,alcaldesa cartagena ana belen castejon rojo ju...,1,0
442,los votantes vox aunque ideologia muy conserva...,1,0
550,las flatulencias decir los gases que crean nue...,0,0


In [52]:
from sklearn.metrics import accuracy_score

print('Exactitud: ', accuracy_score(y_test, y_pred))

Exactitud:  0.7383333333333333
